In [1]:
from pyscipopt import Model, quicksum
import numpy as np

In [2]:
from pyscipopt import Model, Eventhdlr, quicksum, SCIP_EVENTTYPE

INT_TOL = 1e-6

def _safe_set_param(model, name, value):
    try:
        if hasattr(model, "hasParam") and model.hasParam(name):
            if isinstance(value, (bool, int)): model.setIntParam(name, int(value)); return
            if isinstance(value, float):        model.setRealParam(name, float(value)); return
            if isinstance(value, str):          model.setStringParam(name, value); return
        try: model.setIntParam(name, int(value)); return
        except: pass
        try: model.setRealParam(name, float(value)); return
        except: pass
        try: model.setStringParam(name, str(value)); return
        except: pass
    except:
        pass

def _is_integral_from_curr_lp(model, vars_):
    for v in vars_:
        if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
            val = model.getSolVal(None, v)
            if abs(val - round(val)) > INT_TOL:
                return False
    return True

class NodeTracker(Eventhdlr):
    def __init__(self):
        super().__init__()
        self.active = {}
        self.sols = []
        self.allvars = None

    def eventinit(self):
        for et in [
            SCIP_EVENTTYPE.NODEBRANCHED,
            SCIP_EVENTTYPE.NODEFOCUSED,
            SCIP_EVENTTYPE.LPSOLVED,
            SCIP_EVENTTYPE.NODEINFEASIBLE,
            SCIP_EVENTTYPE.NODEFEASIBLE,
            SCIP_EVENTTYPE.NODESOLVED,
            SCIP_EVENTTYPE.BESTSOLFOUND,
        ]:
            try: self.model.catchEvent(et, self)
            except: pass
        self.allvars = self.model.getVars()

    def _mark_open(self, node):
        try:
            nid = node.getNumber(); depth = self.model.getDepth()
        except: 
            return
        if nid not in self.active:
            self.active[nid] = {"status": "open", "depth": depth, "lp_bound": None, "integral": None}

    def _close(self, node, reason):
        try: nid = node.getNumber()
        except: return
        if nid in self.active: self.active[nid]["status"] = reason

    def eventexec(self, event):
        et = event.getType()

        if et == SCIP_EVENTTYPE.NODEBRANCHED:
            node = getattr(event, "getNode", lambda: None)() or self.model.getCurrentNode()
            if node: self._mark_open(node)

        elif et == SCIP_EVENTTYPE.NODEFOCUSED:
            node = self.model.getCurrentNode()
            if node: self._mark_open(node)

        elif et == SCIP_EVENTTYPE.LPSOLVED:
            node = self.model.getCurrentNode()
            if not node: 
                return
            self._mark_open(node)
            try: 
                lb = self.model.getLPObjVal()
            except: 
                lb = None
            nid = node.getNumber()
            self.active[nid]["lp_bound"] = lb

            integral = _is_integral_from_curr_lp(self.model, self.allvars)
            self.active[nid]["integral"] = integral
            if integral:
                vals = {}
                for v in self.allvars:
                    if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
                        vals[v.name] = int(round(self.model.getSolVal(None, v)))
                try: obj = self.model.getLPObjVal()
                except: obj = None
                self.sols.append({"obj": obj, "values": vals})
                self._close(node, "integer")

        elif et == SCIP_EVENTTYPE.NODEINFEASIBLE:
            node = getattr(event, "getNode", lambda: None)() or self.model.getCurrentNode()
            if node: self._close(node, "infeasible")

        elif et == SCIP_EVENTTYPE.BESTSOLFOUND:
            sol = self.model.getBestSol()
            if sol:
                try: obj = self.model.getSolObjVal(sol)
                except: obj = None
                vals = {}
                for v in self.allvars:
                    if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
                        vals[v.name] = int(round(self.model.getSolVal(sol, v)))
                self.sols.append({"obj": obj, "values": vals})

from pyscipopt import SCIP_PARAMSETTING

def solve_ip_enumerate_like(build_model_fn, node_limit, time_limit=None, force_branch=True):
    """
    Run B&B with a node limit, log integer-feasible solutions via NodeTracker,
    and return the true active node list using model.getOpenNodes().

    Set force_branch=True to disable presolve/heuristics/separating so root doesn't close.
    """
    # build random model
    model = build_model_fn()

    # disable heuristics/presolve/separating to avoid root closing
    if force_branch:
        model.setHeuristics(SCIP_PARAMSETTING.OFF)   # disable primal heuristics
        model.setPresolve(SCIP_PARAMSETTING.OFF)     # disable presolve
        model.setSeparating(SCIP_PARAMSETTING.OFF)   # disable cut separation

    # Stop rules 
    model.setLongintParam("limits/nodes", node_limit)
    if time_limit is not None:
        model.setRealParam("limits/time", float(time_limit))

    # attach tracker
    tracker = NodeTracker()
    model.includeEventhdlr(tracker, "NodeTracker", "Log integer nodes")

    model.optimize()

    # get open nodes
    leaves, children, siblings = model.getOpenNodes()
    def _node_info(n):
        try:
            lb = n.getLowerbound()
        except Exception:
            lb = None
        return {
            "node_id": n.getNumber(),
            "parent_id": (n.getParent().getNumber() if n.getParent() else None),
            "depth": n.getDepth(),
            "lower_bound": lb,
            "is_active": n.isActive(),
        }
    active_nodes = [_node_info(n) for n in (leaves + children + siblings)]

    try:
        nodes_remaining = model.getNNodesLeft()
    except AttributeError:
        nodes_remaining = None

    out = {
        "status": model.getStatus(),
        "nodes_processed": model.getNNodes(),
        "nodes_remaining": nodes_remaining,
        "solving_time": model.getSolvingTime(),
        "integer_solutions": tracker.sols,   # keep only tracker to avoid double-counting
        "active_nodes": active_nodes,
    }

    try:
        model.free()
    except Exception:
        pass
    return out


In [28]:
# your TSP builder (as given)
def build_tsp_mtz_euclid(n=15, seed=1):
    rng = np.random.default_rng(seed)
    pts = rng.random((n, 2))
    dist = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            if i == j: continue
            d = np.linalg.norm(pts[i] - pts[j])
            dist[i, j] = int(round(100 * d))  # rounded Euclidean

    m = Model("TSP_MTZ")
    x = {(i, j): m.addVar(vtype="B", name=f"x({i},{j})")
         for i in range(n) for j in range(n) if i != j}
    u = {i: m.addVar(lb=0, ub=n-1, vtype="C", name=f"u({i})") for i in range(n)}

    m.setObjective(quicksum(dist[i, j]*x[i, j] for (i, j) in x), "minimize")

    for i in range(n):
        m.addCons(quicksum(x[i, j] for j in range(n) if i != j) == 1)
    for j in range(n):
        m.addCons(quicksum(x[i, j] for i in range(n) if i != j) == 1)

    m.addCons(u[0] == 0)
    for i in range(1, n): m.chgVarLb(u[i], 1.0)
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                m.addCons(u[i] - u[j] + n*x[i, j] <= n - 1)
    return m

In [13]:
res = solve_ip_enumerate_like(
    build_model_fn=lambda: build_tsp_mtz_euclid(n=5, seed=7),
    node_limit=1000000,
    time_limit=None,
    force_branch=True
)


presolving:
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (0 rounds: 0 fast, 0 medium, 0 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 25 variables (20 bin, 0 int, 0 impl, 5 cont) and 23 constraints
     23 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
  0.0s|     1 |     0 |    12 |     - |   768k |   0 |  25 |  22 |  22 |   0 |  0 |   0 |   0 | 1.834000e+02 |      --      |    Inf | unknown
  0.0s|     1 |     2 |    12 |     - |   786k |   0 |  25 |  22 |  22 |   0 |  1 |   0 |   6 | 1.834000e+02 |      --      |

In [19]:
print("status:", res["status"], "\nnodes processed:", res["nodes_processed"], "\nopen:", len(res["active_nodes"]))
print("# integer solutions seen:", len(res["integer_solutions"]))

status: optimal 
nodes processed: 5 
open: 0
# integer solutions seen: 2


Enumeration Attempt

In [29]:
from pyscipopt import Model, Eventhdlr, SCIP_EVENTTYPE, SCIP_PARAMSETTING

INT_TOL = 1e-6

def _is_integral_from_curr_lp(model, vars_):
    for v in vars_:
        if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
            val = model.getSolVal(None, v)
            if abs(val - round(val)) > INT_TOL:
                return False
    return True

def _node_details(n):
    try:
        pid = n.getParent().getNumber() if n.getParent() else None
    except Exception:
        pid = None
    try:
        lb = n.getLowerbound()
    except Exception:
        lb = None
    return {
        "node_id": n.getNumber(),
        "parent_id": pid,
        "depth": n.getDepth(),
        "lower_bound": lb,
    }

class FullEnumTracker(Eventhdlr):
    """
    Logs:
      - integer_solutions: each LP-integral node (with node details + integer solution)
      - infeasible_nodes: node details when LP is infeasible
    """
    def __init__(self):
        super().__init__()
        self.integer_solutions = []
        self.infeasible_nodes = []
        self.allvars = None

    def eventinit(self):
        for et in [
            SCIP_EVENTTYPE.NODEFOCUSED,
            SCIP_EVENTTYPE.LPSOLVED,
            SCIP_EVENTTYPE.NODEINFEASIBLE,
            SCIP_EVENTTYPE.NODESOLVED,
        ]:
            try:
                self.model.catchEvent(et, self)
            except:
                pass
        self.allvars = self.model.getVars()

        # Keep cutoff bound at +inf so SCIP never prunes by bound after an incumbent appears.
        try:
            self.model.setCutoffbound(float('inf'))
        except:
            pass

    def eventexec(self, event):
        et = event.getType()

        if et == SCIP_EVENTTYPE.LPSOLVED:
            node = self.model.getCurrentNode()
            if not node:
                return

            # maintain +inf cutoff continuously
            try:
                self.model.setCutoffbound(float('inf'))
            except:
                pass

            # if LP solution is integral, capture and we're done with this node
            if _is_integral_from_curr_lp(self.model, self.allvars):
                vals = {}
                for v in self.allvars:
                    if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
                        vals[v.name] = int(round(self.model.getSolVal(None, v)))
                try:
                    obj = self.model.getLPObjVal()
                except Exception:
                    obj = None

                nd = _node_details(node)
                nd.update({"obj": obj, "values": vals})
                self.integer_solutions.append(nd)

        elif et == SCIP_EVENTTYPE.NODEINFEASIBLE:
            node = getattr(event, "getNode", lambda: None)() or self.model.getCurrentNode()
            if node:
                self.infeasible_nodes.append(_node_details(node))

        elif et == SCIP_EVENTTYPE.NODESOLVED:
            # no-op; enumeration continues automatically until no open nodes remain
            pass

def solve_ip_full_enumeration(build_model_fn, time_limit=None, force_branch=True):
    """
    Run B&B until the tree is exhausted, with NO node limit and NO bound-based pruning.
    Leaves must end as either LP-infeasible or LP-integral.
    Integer nodes are logged with node details and integer solution.

    Returns:
      {
        status, nodes_processed, solving_time,
        integer_solutions: [ {node_id, parent_id, depth, lower_bound, obj, values{var:int}} ... ],
        infeasible_nodes:  [ {node_id, parent_id, depth, lower_bound} ... ],
        open_nodes_final:  [ ... ]   # should be empty if tree exhausted
      }
    """
    model = build_model_fn()

    if force_branch:
        model.setHeuristics(SCIP_PARAMSETTING.OFF)
        model.setPresolve(SCIP_PARAMSETTING.OFF)
        model.setSeparating(SCIP_PARAMSETTING.OFF)
        # optional: reduce extra pruning/propagation “cleverness”
        try: model.setParam("propagating/maxrounds", 0)
        except: pass

    # NO node limit
    # (do not set limits/nodes)
    if time_limit is not None:
        model.setRealParam("limits/time", float(time_limit))

    # Make sure no cutoff pruning ever happens (even after an incumbent is found)
    try:
        model.setCutoffbound(float('inf'))
    except:
        pass

    tracker = FullEnumTracker()
    model.includeEventhdlr(tracker, "FullEnumTracker", "Enumerate: only LP-infeasible or LP-integral leaves")

    model.optimize()

    # gather final open nodes (should be empty if fully exhausted)
    leaves, children, siblings = model.getOpenNodes()
    def _ni(n):
        return {
            "node_id": n.getNumber(),
            "parent_id": (n.getParent().getNumber() if n.getParent() else None),
            "depth": n.getDepth(),
            "lower_bound": (n.getLowerbound() if hasattr(n, "getLowerbound") else None),
            "is_active": n.isActive(),
        }
    open_nodes_final = [_ni(n) for n in (leaves + children + siblings)]

    out = {
        "status": model.getStatus(),
        "nodes_processed": model.getNNodes(),
        "solving_time": model.getSolvingTime(),
        "integer_solutions": tracker.integer_solutions,
        "infeasible_nodes": tracker.infeasible_nodes,
        "open_nodes_final": open_nodes_final,
    }

    try: model.free()
    except: pass
    return out


In [34]:
results = solve_ip_full_enumeration(
    build_model_fn=lambda: build_tsp_mtz_euclid(n=15, seed=10),
    time_limit=None,       # optional; seconds
    force_branch=True    # ensures presolve/cuts/heuristics are off
)


presolving:
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (0 rounds: 0 fast, 0 medium, 0 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 225 variables (210 bin, 0 int, 0 impl, 15 cont) and 213 constraints
    213 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
  0.0s|     1 |     0 |    40 |     - |  2312k |   0 | 225 | 212 | 212 |   0 |  0 |   0 |   0 | 2.382000e+02 |      --      |    Inf | unknown
  0.0s|     1 |     2 |    40 |     - |  2344k |   0 | 225 | 212 | 212 |   0 |  1 |   0 |  24 | 2.382667e+02 |      --   

In [35]:
print("Status:", results["status"])
print("Nodes processed:", results["nodes_processed"])
print("Integer solutions found:", len(results["integer_solutions"]))
print("Infeasible nodes:", len(results["infeasible_nodes"]))
print("Remaining open nodes:", len(results["open_nodes_final"]))

# Inspect one integer node:
if results["integer_solutions"]:
    print("\nExample integer solution node:")
    print(results["integer_solutions"][0])


Status: optimal
Nodes processed: 14971
Integer solutions found: 261
Infeasible nodes: 14554
Remaining open nodes: 0

Example integer solution node:
{'node_id': 38, 'parent_id': 36, 'depth': 10, 'lower_bound': 1e+20, 'obj': 1e+20, 'values': {'x(0,1)': 999999999999999967336168804116691273849533185806555472917961779471295845921727862608739868455469056, 'x(0,2)': 999999999999999967336168804116691273849533185806555472917961779471295845921727862608739868455469056, 'x(0,3)': 999999999999999967336168804116691273849533185806555472917961779471295845921727862608739868455469056, 'x(0,4)': 999999999999999967336168804116691273849533185806555472917961779471295845921727862608739868455469056, 'x(0,5)': 999999999999999967336168804116691273849533185806555472917961779471295845921727862608739868455469056, 'x(0,6)': 999999999999999967336168804116691273849533185806555472917961779471295845921727862608739868455469056, 'x(0,7)': 99999999999999996733616880411669127384953318580655547291796177947129584592172786260